In [20]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import pathlib
import spacy
from spacy.lang.en import English
from spacy.lang.en.stop_words import STOP_WORDS
from string import punctuation
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras import layers
import keras

In [21]:
# Loading the dataset 
data_dir = pathlib.Path('/home/lv11/Documents/ProyectosPython/sentimentAnalysis/train')
nf = pd.read_csv(data_dir / 'tweetsDataset1.csv',skiprows=1,names=['Message','Target'])
print(nf.head(20))

nlp = English()
stop_words = list(STOP_WORDS)
print(stop_words)

def spacy_tokenizer(sentence):
    tokens = nlp(sentence)
    tokens = [ word.lemma_.lower().strip() if word.lemma_ != '-PRON-' else word.lower_ for word in tokens ]
    tokens = [ word for word in tokens if word not in stop_words and word not in punctuation ]
    return tokens

Message  Target
0                                             Hi Guys       1
1             it's just got 3 years longer unhappy          0
2   Hey thanks for being top new followers this we...       1
3                                 I know how you feel       0
4                                          huh? happy       1
5                             Didn't keep it unhappy        0
6                                These are for  happy       1
7   unhappy  how come people like this have childr...       0
8                                  Bapak Tom Cruise.        0
9   help me too unhappy  as an early birthday gift...       0
10  Shoutout to for the donation. Looks like I'm s...       1
11                             Good luck Swampy happy       1
12  lets agree she put her brain on knees crying w...       1
13                                         Nitpicking       1
14                 unhappy  :/ unhappy   feeling sick       0
15                      which is good for him happy.1 

In [24]:
vectorizer = CountVectorizer(min_df=0, lowercase=False)
vectorizer.fit(nf['Message'])
print(vectorizer.vocabulary_)

vectorizer.transform(nf['Message']).toarray()

messages = nf['Message']
labels = nf['Target']

3353, 'abeg': 1609, 'inspiring': 2894, 'visit': 4389, 'Luxor': 865, 'city': 2064, 'Egypt': 460, 'pleas': 3524, 'computer': 2118, 'failed': 2473, 'Even': 474, 'UE4': 1461, 'tr': 4251, 'confining': 2127, 'zoos': 4564, 'Beetle': 261, 'joins': 2959, 'Wall': 1522, 'gigs': 2645, 'rusty': 3759, 'Haven': 644, 'played': 3520, 'kept': 2997, 'crocodile': 2182, 'iris': 2917, 'missions': 3251, 'heroic': 2783, 'dungeons': 2353, 'japanese': 2938, 'Pixel': 1105, 'germany': 2636, 'authority': 1784, 'countries': 2161, 'Okey': 1035, 'Megan': 917, 'jittery': 2954, 'caffeine': 1974, 'cad': 1971, 'Quebec': 1138, 'taxes': 4150, 'DIGITAL': 396, 'MONSTERS': 884, 'WANT': 1501, 'WORK': 1513, 'BLACKPINK': 229, 'IT': 693, 'HAPPEN': 608, 'READING': 1142, 'HOPE': 623, 'SOMETHING': 1221, 'GREAT': 552, 'HAPPENS': 609, 'bet': 1854, 'sided': 3902, 'Tim': 1423, 'Tams': 1383, 'surprisingly': 4102, 'alt': 1678, 'roar': 3736, 'droll': 2338, 'Fad': 502, 'titties': 4219, 'fantastic': 2486, 'Kissy': 789, 'offering': 3377, 'sea

In [33]:
x_train, x_test, y_train, y_test = train_test_split(messages,labels,test_size=0.25,random_state=1000,shuffle=True)

len(x_train)

1727

In [26]:
vectorizer.fit(x_train)
X_train = vectorizer.transform(x_train).toarray()
X_test = vectorizer.transform(x_test).toarray()

print(X_train)
print(X_test)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [34]:
features = 1727
shufle = True
drop = 0.5
layer1 = 512
layer2 = 256
epoch = 5
lr = 0.002
epsilon = None
validation = 0.1

In [35]:
model = Sequential()
model.add(layers.Dense(layer1, input_dim=features,activation='relu'))
model.add(layers.Dropout(drop))
model.add(layers.Dense(layer2,activation='sigmoid'))
model.add(layers.Dropout(drop))
model.add(layers.Dense(3,activation='softmax'))

In [36]:
optimizer = keras.optimizers.Adam(lr=lr, beta_1=0.9, beta_2=0.999, epsilon=epsilon,decay=0.0,amsgrad=False)

In [37]:
model.compile(loss='sparse_categorical_crossentropy',optimizer=optimizer,metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 512)               884736    
_________________________________________________________________
dropout_3 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 256)               131328    
_________________________________________________________________
dropout_4 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 3)                 771       
Total params: 1,016,835
Trainable params: 1,016,835
Non-trainable params: 0
_________________________________________________________________


In [38]:
model.fit(np.array(X_train),y_train,batch_size=32,epochs=epoch,verbose=1,validation_split=validation,shuffle=shufle)

ValueError: Error when checking input: expected dense_6_input to have shape (1727,) but got array with shape (3749,)